****************************************************************

# Data exploration

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


### Quick-note on project directory

The main root dir `~/3dcorrection` is structured as follow:
* `data/` contains raw and preprocessed data. 
    * `raw/` is actually a symbolic link to the same repo for all candidates, DO NOT TOUCH IT!
    * `processed/` will be created when data is preprocessed and will contain all transformed data
* 

In [1]:
import os
import os.path as osp

root_path = osp.join('/', 'root', 'bootcamps')

data_path = osp.join(root_path, 'data')
cache_path = osp.join(data_path, 'cache')
raw_data_path = osp.join(data_path, 'raw')
processed_data_path = osp.join(data_path, 'processed')

### The 3D Correction Use-Case

The European Centre for Medium-range Weather Forecasts (ECMWF) has developed a series of model giving the current best accurate parametrization scheme available—among those, SPARTACUS delivers **radiation** prediction over the globe. Because it is demanding in computations, a simpler, degraded model called TRIPLECLOUD is developed to satisfy the production environment constraints. 

Like most climate models, to leverage hardware acceleration, the choice is made to split the globe in blocks—this has the immediate consequence of losing the spatial correlation for a gain in parallelization. 

The unit block is a column that express values throughout the vertical dimension over a set of levels. Each level is

Now let's load the raw data we'll be using throughout this hands-on. Take a look at the [source notebook](https://git.ecmwf.int/projects/MLFET/repos/maelstrom-radiation/browse/climetlab_maelstrom_radiation/radiation.py) for a more info on the variables.

In [ ]:
import climetlab as cml
import dask
import dask.array as da
from glob import glob
import numpy as np
import os.path as osp
import time
import xarray as xr

import config

step = 250

feats_path = osp.join(processed_data_path, f'feats-{step}', 'concat')
feats = da.from_npy_stack(feats_path)

# np.load(osp.join(processed_data_path

In [ ]:
x = feats[..., :20]
y = feats[..., 20:]

In [3]:
x

dask.array<from-npy-stack-/root/bootcamps/data/processed/feats-250, shape=(4070400, 138, 20), dtype=float32, chunksize=(9600, 138, 20), chunktype=numpy.ndarray>

In [4]:
n = 2
x_np = np.concatenate((x.compute(),) * n, axis=-1)
y_np = np.concatenate((y.compute(),) * n, axis=-1)

In [5]:
s = time.perf_counter()
x_mean_np = np.mean(x_np, axis=0)
y_mean_np = np.mean(y_np, axis=0)
x_std_np = np.std(x_np, axis=0)
y_std_np = np.std(y_np, axis=0)
e = time.perf_counter()

print("%.2f ms" % ((e - s) * 1000))

711180.64 ms


In [6]:
x_mean = da.mean(x, axis=0)
y_mean = da.mean(y, axis=0)
x_std = da.std(x, axis=0)
y_std = da.std(y, axis=0)

In [7]:
s = time.perf_counter()
x_mean.compute()
y_mean.compute()
x_std.compute()
y_std.compute()
e = time.perf_counter()

print("%.2f ms" % ((e - s) * 1000))

13747.57 ms


In [6]:
x

dask.array<from-npy-stack, shape=(1085440, 138, 20), dtype=float32, chunksize=(2560, 138, 20), chunktype=numpy.ndarray>

In [7]:
y

dask.array<from-npy-stack, shape=(1085440, 138, 4), dtype=float32, chunksize=(2560, 138, 4), chunktype=numpy.ndarray>

The returned object is a ClimateLab dataset Xarray Dataset

Let's check the content of the downloaded file

most operations are computed lazily in dask/xarray when needed and if possible on every chunk, treated and seen 'as if' it was a continuous array

In [9]:
import torch

stats_path = osp.join(processed_data_path, f"stats-{config.params['timestep']}.pt")
s = time.perf_counter()
stats = {
    "x_mean" : torch.tensor(x_mean.compute()),
    "y_mean" : torch.tensor(y_mean.compute()),
    "x_std" : torch.tensor(x_std.compute()),
    "y_std" : torch.tensor(y_std.compute())
}
e = time.perf_counter()
torch.save(stats, stats_path)